# Gold Layer

## Configuração do servidor para trabalho no Gold Layer

In [0]:
from os import listdir
import uuid
import gc
import urllib.request
from pyspark.sql.functions import *
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, TimestampType, LongType, DateType, TimestampType, BooleanType, DoubleType 

# Create a SparkSession with the required configurations for Delta Lake
spark = SparkSession.builder \
    .appName("Carga Delta") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

## Funções

### Funções para verificar o id_job e caminhos.
Com execeção dos paths dentro do schema gold, qualquer erro levantado nas funções abaixo é com relação a processos de outras camadas

In [0]:
# verifica se os caminhos para a camada gold existem
def gold_volume():
    try:
        spark.sql("create schema if not exists data_ex.gold")
        if not spark.sql("show volumes in data_ex.gold").collect():
            spark.sql(f"create volume if not exists data_ex.gold.main")
            print("Volume criado com sucesso.")
            return 1
        else:
            print("Volume já existe.")
            return 1
    except Exception as e:
        print(f"Ocorreu um erro na função gol_directories(): {e}") 
        return 0

# verifica se os caminhos para a camada silver existem
def exist_silver_path(volume):
    volume = volume.lower()
    try:
        command = spark.sql(f"show volumes in data_ex.silver").collect() 
        if not command:  
            print(f"Não existem volumes na camada silver")
            return 0
        else:
            if command[0][1] == volume:
                print(f"O volume {volume} existe na camada silver")
                return 1
            print(f"O volume {volume} não existe na camada silver data_ex.silver")
            return 0
    except Exception as e:
        print(f"Ocorreu um erro na função exist_silver_path(): {e})")
        return 0

# Verifica de o id_job existe no metadados
def check_jo_id(camada):
    relacoes = {
        "bronze" : "meta_bronze",
        "silver" : "meta_silver",
        "gold" : "meta_gold"
    }
    try:
        if not spark.sql(f"select * from data_ex.metadados.{relacoes[camada]}"):
            print(f"id_job para a camada {camada} inesiste, gere um novo id_job para a camada {camada}")
            return 0
        else:
            print(f"id_job para a camada {camada} existe")
            return 1
    except Exception as e:
        print(f"Ocorreu um erro na função check_jo_id(): {e})")
        return 0

### (df || 0) read(path, volume)

In [0]:
def read(volume, file_name):
    try:
        df = spark.read\
            .format("delta")\
            .load(f"/Volumes/data_ex/silver/{volume}/{file_name}")
        return df
    except Exception as e:
        print(f"Erro ao tentar ler {file_name}: {e}")
        return 0

### Funções de log de carga

In [0]:
def make_log_tabel(camada):
    relacoes = {
            "bronze" : "bronze_log_carga",
            "silver" : "silver_log_carga",
            "gold"   : "gold_log_carga"
        }
    n = 0
    # Conta quantas tabelas existem
    try:
        n = spark.sql("show tables in data_ex.bronze").count()
    except:
        print("ERRO ao ler a tabela de logs")
    # Cria a tabela se não existir 
    if n == 0:
        try :
            spark.sql(f"""create table if not exists data_ex.bronze.{relacoes[camada]} (
                id_carga string,
                id_job string,
                nome_arquivo string,
                fonte string,
                camada string,
                path_origem string,
                path_destino string,
                data_inicio timestamp,
                data_fim timestamp,
                duracao_ms string,
                registros_lidos integer,
                registros_gravados integer,
                status string,
                mensagem_erro string,
                data_execusao date
                ) using delta """)
            return 1
        except Exception as e:
            print(f"Erro ao criar a tabela de logs: {e}")
            return 0
    # Erro, existe mais de uma tabela
    elif n > 1:
        print(f"Erro na árvore de diretórios, exite mais de {n} tabelas de losgs na camada bronze")
        return 0
    # A tabela ja existe
    else:
        print("Tabela de logs ja existes na camada bronze")
        return 1

In [0]:
def generate_log(volume, camada_destino, file, new_name):
  camada_origem = "silver"
  try:
    global log_data
    log_data.update({
      "id_carga" : str(uuid.uuid4()),
      "id_job"   : spark.sql("select * from data_ex.metadados.meta_bronze").collect()[0][0],
      "nome_arquivo" : new_name,
      "fonte" : "filesystem_local",
      "camada" : "bronze",
      "path_origem" : f"/Volumes/data_ex/{camada_origem}/{volume}/{file}",
      "path_destino" : f"/Volumes/data_ex/{camada_destino}/{volume}/{new_name}",
      "data_inicio" : spark.range(1).select(current_timestamp()).collect()[0][0],
      "status" : "Running",
      "data_execusao" : spark.range(1).select(current_date()).collect()[0][0]
    })
    return 1
  except Exception as e:
    return 0

In [0]:
def update_log():
    try:
        global log_data
        log_data["status"] = "True"   # Atualiza o status do log
        data_fim = spark.range(1).select(current_timestamp()).collect()[0][0]
        log_data["data_fim"] = data_fim  
        duracao_ms = (data_fim - log_data[7]).total_seconds() * 1000
        log_data["duracao_ms"] = duracao_ms
        acao = storeLog(log_data)
        if acao == 1:
            return 1
        else:
            return 0 
    except Exception as e:
        print(f"Erro na funcao gerarLog(): {e}")

In [0]:
def storeLog():
    try:
        global log_data
        row = Row(
            id_carga = log_data["id_carga"],
            id_job = log_data["id_job"],
            nome_arquivo = log_data["nome_arquivo"],
            fonte = log_data["fonte"],
            camada = log_data["camada"],
            path_origem = log_data["path_origem"],
            path_destino = log_data["path_destino"],
            data_inicio = log_data["data_inicio"],
            data_fim = log_data["data_fim"],
            duracao_ms = log_data["duracao_ms"],
            registros_lidos = log_data["registros_lidos"],
            registros_gravados = log_data["registros_gravados"],
            status = log_data["status"],
            mensagem_erro = log_data["mensagem_erro"],
            data_carga = log_data["data_carga"]
        )

        df_log = spark.createDataFrame([row])
        df_log.write.mode("append").insertInto("data_ex.bronze.bronze_log_carga")
        return 1
    except Exception as e:
        print(f"Erro ao gravar o log: {e}")
        return 0

### (df || 0) save(df, new_file_name)

In [0]:
def save_table(df, new_file_name):
    try:
        if "fato" in new_file_name:
            df_data = spark.read.format("delta").load("/Volumes/data_ex/gold/main/dim_data")
            
            df_fato = df\
                .join(broadcast(
                    df_data.select("sk_data", "ano", "mes")),
                      "sk_data")\
            
            df_fato\
                .write\
                .format("delta")\
                .mode("overwrite")\
                .partitionBy("ano","mes")\
                .saveAsTable(f"data_ex.gold.{new_file_name}")
            
            df_fato\
                .write\
                .format("delta")\
                .mode("overwrite")\
                .save(f"/Volumes/data_ex/gold/main/{new_file_name}")
            
            return df_fato
        else:
            df.write\
                .format("delta")\
                .mode("overwrite")\
                .saveAsTable(f"data_ex.gold.{new_file_name}")
            df.write\
                .format("delta")\
                .mode("overwrite")\
                .save(f"/Volumes/data_ex/gold/main/{new_file_name}")
            return df
    except Exception as e:
        print(f"Ocorreu um erro na função save(): {e}")
        return 0

### execute_gold_layer()

In [0]:
# Lista de logs
log_data = {}

def execute_gold_layer(volume, camada, new_files_names):

    # Garante que as pastas no unity catalog gold existam
    if not gold_volume():
        return

    # Garante a existencia da tabelas de logs de acesso da camada gold
    if not make_log_tabel(camada):
        return

    # Verifica se existe o caminho até a camada silver
    if not exist_silver_path(volume):
        return
    
    # Verifica se existe o id_job no metadados
    if not check_jo_id(camada):
        return
    
    files = listdir("/Volumes/data_ex/silver/download001")
    
    for i, (file, new_file_name) in enumerate(zip(files,new_files_names)): 

        # Gerar o log
        generate_log(volume, camada, file, new_file_name)

        # Ler o arquivo
        df_read = read(volume, file)
        
        # df_read e nulo ou 0
        if not df_read:
            return
        
        # df_read é um dataframe nao nulo, assim, podemos contar os registros lidos
        global log_data
        log_data["registros_lidos"] = df_read.count()

        # Salvar o df_read na camada gold
        df_read = save_table(df_read, new_file_name)

        # df_read e nulo ou 0
        if not df_read:
            return
        
        # Conta quantos registros foram gravados
        log_data["registros_gravados"] = df_read.count()
        
        # Atualiza o log
        if not update_log:
            return
        
        print(f"Arquivo {file} processado com sucesso.")

        
        


# Main

In [0]:
volume = "download001"
camada = "gold"

new_files_names = [
    "dim_categoria_produto",
    "dim_cliente",
    "dim_data", 
    "dim_localidade",
    "dim_produto",
    "fato_vendas"
] 

execute_gold_layer(volume, camada, new_files_names)

# Impressão dos resultados

In [0]:
%skip

volume = "download001"
camada = "gold"

new_files_names = [
    "gold_dim_categoria_produto",
    "gold_dim_cliente",
    "gold_dim_data", 
    "gold_dim_localidade",
    "gold_dim_produto",
    "gold_fato_vendas"
] 

def see_tables(camada, new_file_names):
    for file in new_files_names:
        df = spark.read.format("delta").load(f"/Volumes/data_ex/{camada}/main/{file}")
        display(df)

see_tables("gold", new_files_names)


# Salvando como table para exportação

# Limpar Memória

In [0]:
import gc

gc.collect()

# Reset

In [0]:
# spark.sql("drop schema if exists data_ex.gold cascade")